In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder

In [33]:
xgb_df = pd.read_csv('C:/python/signate/data/xgb_pred_proba.csv',index_col=0)
xgb_test_df = pd.read_csv('C:/python/signate/data/xgb_pred_proba_test.csv',index_col=0)
knn_df = pd.read_csv('C:/python/signate/data/knn_pred.csv', index_col=0)
knn_test_df = pd.read_csv('C:/python/signate/data/knn_predict_test.csv', index_col=0)
lgb_df = pd.read_csv('C:/python/signate/data/lgb_pred_proba.csv', index_col=0)
lgb_test_df = pd.read_csv('C:/python/signate/data/lgb_pred_proba_test.csv', index_col=0)
train_df = pd.read_csv('C:/python/signate/data/train.csv', index_col=0)
sample_df = pd.read_csv('C:/python/signate/data/sample_submission.csv', header=None, index_col=0)
dive_test_lgb = pd.read_csv('C:/python/signate/data/lgb-dive.csv', index_col=0, header=None)
dive_test_lgb.rename(columns={1:'lgb'}, inplace=True)
dive_test_xgb = pd.read_csv('C:/python/signate/data/xgb-dive.csv', index_col=0, header=None)
dive_test_xgb.rename(columns={1:'xgb'}, inplace=True)
lgb_dive_train = pd.read_csv('C:/python/signate/data/lgb_dive-train.csv', index_col=0, )
xgb_dive_train = pd.read_csv('C:/python/signate/data/xgb_dive-train.csv', index_col=0, )

df = pd.concat([xgb_df, knn_df, lgb_df, lgb_dive_train, xgb_dive_train], axis=1)
test_df = pd.concat([xgb_test_df, knn_test_df, lgb_test_df, dive_test_lgb, dive_test_xgb], axis=1)

In [38]:
df.head(30)

,xgb-0,xgb-1,xgb-2,knn_predict,lgb-0,lgb-1,lgb-2,lgb,xgb
0,0.611466,0.279555,0.108979,0,0.470961,0.393190,0.135849,0,0
1,0.822389,0.173742,0.003870,1,0.313045,0.678275,0.008681,1,1
2,0.472740,0.248011,0.279249,1,0.196880,0.316276,0.486844,1,0
3,0.117261,0.521242,0.361498,0,0.056534,0.913599,0.029867,1,1
4,0.341689,0.430007,0.228304,0,0.361365,0.612840,0.025794,1,1
5,0.021359,0.961191,0.017450,1,0.045343,0.873851,0.080806,1,1
6,0.366866,0.557717,0.075416,1,0.653244,0.260746,0.086010,0,0
7,0.077631,0.711424,0.210944,1,0.125549,0.857428,0.017023,1,1
8,0.244491,0.695453,0.060057,1,0.176771,0.816259,0.006970,1,1
9,0.558481,0.418806,0.022714,1,0.770695,0.215841,0.013465,0,0


In [35]:
test_df

,xgb-0,xgb-1,xgb-2,knn_predict,lgb-0,lgb-1,lgb-2,lgb,xgb
19984,0.545216,0.404843,0.049941,1,0.596886,0.363536,0.039578,0,0
19985,0.362207,0.587780,0.050013,1,0.421671,0.570713,0.007616,1,1
19986,0.255562,0.733134,0.011305,1,0.637548,0.353731,0.008721,0,1
19987,0.337109,0.617176,0.045715,1,0.201457,0.789298,0.009245,1,1
19988,0.480037,0.468837,0.051126,1,0.585373,0.382282,0.032345,0,0
...,...,...,...,...,...,...,...,...,...
39964,0.045060,0.944733,0.010206,0,0.103593,0.889922,0.006485,1,1
39965,0.358874,0.481291,0.159836,1,0.367410,0.599118,0.033472,1,1
39966,0.107937,0.139446,0.752617,2,0.034041,0.155305,0.810654,1,1
39967,0.339700,0.639044,0.021256,0,0.254084,0.737013,0.008903,1,1


In [11]:
# 多数決

lgb_test_df.rename(columns={'lgb-0':0, 'lgb-1':1, 'lgb-2':2}, inplace=True)
xgb_test_df.rename(columns={'xgb-0':0, 'xgb-1':1, 'xgb-2':2}, inplace=True)
xgb_test_df_max = pd.DataFrame(xgb_test_df.idxmax(axis=1))
lgb_test_df_max = pd.DataFrame(lgb_test_df.idxmax(axis=1))

# knn_test_dfをonehotに変換
onhot_enc = OneHotEncoder()
knn_test_df_max_onehot = onhot_enc.fit_transform(knn_test_df).toarray()
dive_test_df_onehot = onhot_enc.fit_transform(dive_test_df).toarray()
xgb_test_df_max_onehot = onhot_enc.fit_transform(xgb_test_df_max).toarray()
lgb_test_df_max_onehot = onhot_enc.fit_transform(lgb_test_df_max).toarray()
ensam_df  = lgb_test_df_max_onehot + knn_test_df_max_onehot + dive_test_df_onehot #+ xgb_test_df_max_onehot
ensam_df_max = np.argmax(ensam_df, axis=1)
sample_df[1] = ensam_df_max
sample_df.to_csv('C:/python/signate/data/ensam_lgb-knn-dive.csv', header=None)

In [17]:
xgb_test_df_max

,0
19984,0
19985,1
19986,1
19987,1
19988,0
...,...
39964,1
39965,1
39966,2
39967,1


In [23]:
# 0以外があるなら、その値を採用
dive_test_df.rename(columns={1:'dive-0'}, inplace=True)
xgb_test_df_max.rename(columns={0:'xgb-0'}, inplace=True)
lgb_test_df_max.rename(columns={0:'lgb-0'}, inplace=True)
knn_test_df.rename(columns={1:'knn-0'}, inplace=True)

df = pd.concat([xgb_test_df_max, knn_test_df, lgb_test_df_max, dive_test_df], axis=1)


In [31]:
# probaの最大値

lgb_test_df.rename(columns={'lgb-0':0, 'lgb-1':1, 'lgb-2':2}, inplace=True)
xgb_test_df.rename(columns={'xgb-0':0, 'xgb-1':1, 'xgb-2':2}, inplace=True)
sum_df = xgb_test_df*0.8 + lgb_test_df

sum_df_max = np.argmax(sum_df.values, axis=1)
sample_df[1] = sum_df_max
# sample_df.to_csv('C:/python/signate/data/sum.csv', header=None)

In [37]:
# lgbでスタッキング
le_col = ['knn_predict', 'lgb', 'xgb', ]

df[le_col] = df[le_col].astype('category')
test_df[le_col] = test_df[le_col].astype('category')

y = train_df['health']
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
valid_scores = []
models = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(df, y)):
    print(f'fold{fold} start')
    X_train, y_train = df.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = df.iloc[valid_idx], y.iloc[valid_idx]

    lgb_train = lgb.Dataset(X_train, y_train,)
    lgb_eval = lgb.Dataset(X_valid, y_valid,)


    verbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される
    params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'seed': 0,}
    
    model = lgb.train(params,
                      lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=1000,
                       callbacks=[lgb.early_stopping(stopping_rounds=50, 
                                verbose=True), # early_stopping用コールバック関数
                           lgb.log_evaluation(verbose_eval)], # コマンドライン出力用コールバック関数))
                        # feval=f1,
                        # categorical_feature=le_columns,
                    )

    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_max = np.argmax(y_pred, axis=1)
    score = f1_score(y_valid, y_pred_max, average='macro')
    print(f'fold{fold} f1_score: {score:.4f}')
    valid_scores.append(score)
    models.append(model)
print(f'CV: {np.mean(valid_scores):.4f}')


fold0 start
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001957 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1542
[LightGBM] [Info] Number of data points in the train set: 15987, number of used features: 9
[LightGBM] [Info] Start training from score -1.732206
[LightGBM] [Info] Start training from score -0.238079
[LightGBM] [Info] Start training from score -3.353382
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	training's multi_logloss: 0.586765	valid_1's multi_logloss: 0.606725
fold0 f1_score: 0.2939
fold1 start
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000631 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1542
[LightGBM] [Info] Number of data points in the train set: 15987, num